In [1]:
from google.cloud import bigquery as bq
# from google.cloud.bigquery import LoadJobConfig
# from google.cloud.bigquery import SchemaField
import os
import pandas as pd
import pandas_gbq as pgbq
import util_functions as util
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext google.cloud.bigquery

service_credentials = 'Service_Credentials/big-query-horse-play-f37757d450b8.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_credentials

bq_client = bq.Client()

In [ ]:
util.get_dataset_ids(bq_client)

In [16]:
%%bigquery birth_df

SELECT
    source_year AS year,
    mother_age AS mom_age,
    COUNT(cigarettes_per_day) AS cigs_per_day
FROM `bigquery-public-data.samples.natality`
GROUP BY year, mom_age
ORDER BY year DESC
LIMIT 5

,year,mom_age,cigs_per_day
0,2008,15,323
1,2008,28,7346
2,2008,47,7
3,2008,31,4529
4,2008,12,0


In [ ]:
birth_df[['mom_age', 'cigs_per_day']].groupby(by='mom_age').mean().plot(kind='bar')

In [17]:
# But the above is the same as this
sql_query = (
"""
SELECT
    source_year AS year,
    mother_age AS mom_age,
    COUNT(cigarettes_per_day) AS cigs_per_day
FROM `bigquery-public-data.samples.natality`
GROUP BY year, mom_age
ORDER BY year DESC
LIMIT 5
""")

query_job = bq_client.query(sql_query)

query_results = query_job.result()

# But then you need to process it extensively to get it into a pandas df

In [18]:
# So if you want the best of both worlds, you can just use the pgbq functions rather than line magic
pgbq.read_gbq(sql_query, dialect="standard")

,year,mom_age,cigs_per_day
0,2008,21,10502
1,2008,16,1061
2,2008,45,54
3,2008,17,2794
4,2008,22,10730


In [22]:
# Okay, let's hit the GA bigquery sample
ga_query = (
"""
SELECT SUM(totals.pageviews) as TotalPageviews
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170101`
""")

pgbq.read_gbq(ga_query, dialect="standard")

,TotalPageviews
0,5362


In [ ]:
# Ok, got data back. Now to pull down the schema details